In [ ]:
from collections import defaultdict
from collections.abc import Iterable
from abc import ABC, abstractmethod
class Iterator(ABC):
    @abstractmethod
    def __next__(self):
        pass
    def __iter__(self):
        return self

class wrapper(Iterator):
    def __init__(self, it):
        self.it = iter(it)
    def __next__(self):
        return next(self.it)

class flatten(Iterator):
    def __init__(self, it):
        self.its = [iter(it)]

    def __next__(self):
        try:
            obj = next(self.its[-1])
            if isinstance(obj, Iterable) and not type(obj) == str:
                self.its.append(iter(obj))
                return next(self)
            else:
                return obj
        except StopIteration:
            self.its.pop()
            if not self.its: raise
        return next(self)

class buffered(Iterator):
    def __init__(self, it):
        self.it = iter(it)
        self.queue = []

    def __next__(self):
        if self.queue:
            return self.queue.pop()
        else:
            return next(self.it)

    def enqueue(self, obj):
        self.queue.append(obj)

class Chat:    
    def __init__(self, doc):
        requesterUsername = doc["requesterUsername"]
        responderUsername = doc["responderUsername"]
        self.requests = [
            Request(
                req,
                requesterUsername=requesterUsername,
                responderUsername=responderUsername
                ) 
            for req in doc["requests"]
            ]
        
    def render(self):
        return map(
            lambda line: line + "\n",
            flatten(
                map(
                    lambda req: req.render(),
                    self.requests
                )
            )
        )
    
class VariableData:
    def __init__(self, doc):
        self.variables = []
        for var in doc["variables"]:
            kind = doc.get("kind", None)
            if kind == "file":
                self.variables.append(f"file: `{doc['name']}`")
            else:
                print(f"Unknown variable encountered:\n{doc}")
    def render(self):
        return ", ".join(self.variables)

class Request():
    from arango import ArangoClient
    coll = ArangoClient("http://localhost:8529").db().collection("chat-ids")
    @classmethod
    def get_model(cls, responseId):
        cursor = cls.coll.find({"request_id": responseId}, skip=0, limit=1)
        if cursor.empty(): return None
        else: return cursor.next()["model"]
    
    def __init__(self, doc, *, requesterUsername, responderUsername):
        self.requesterUsername = requesterUsername
        self.responderUsername = responderUsername
        self.responseId = doc["result"]["metadata"]["responseId"]
        self.model = self.get_model(self.responseId)
        self.modes = doc["agent"]["modes"]
        self.message = doc["message"]["text"]
        self.response = Response(doc["response"])
        self.variableData = VariableData(doc["variableData"])

    def render(self):
        return [
            map(
                lambda s: s + "\n",
                [
                    self.requesterUsername + ':',
                    self.message,
                    *([s] if (s:=self.variableData.render()) else []),
                    "",
                    self.responderUsername + (f" ({self.model})" if self.model else "") + ':'   
                ]
            ),
            self.response.render()
        ]
    
class Response:
    def __init__(self, lst):
        self.chunks = []
        it = buffered(lst)
        for chunk in it:
            obj = None
            
            kind = chunk.get("kind", None)
            #tool invocation
            if kind=="toolInvocationSerialized":
                if chunk["toolId"] == "copilot_insertEdit":
                    it.enqueue(chunk)
                    obj = CopilotFileEdit(it)
                
            #text block
            elif (
                ("value" in chunk and kind is None) or
                kind == "inlineReference"
            ):
                it.enqueue(chunk)
                obj = TextBlock(it)
            
            if obj:
                self.chunks.append(obj)
            else:
                print(f"Unknown chunk encountered:\n{chunk}")
    
    def render(self):
        return map(lambda chunk: chunk.render(), self.chunks)

class TextBlock:
    def __init__(self, it):
        self.chunks = []
        for chunk in it:
            kind = chunk.get("kind", None)
            if kind == "inlineReference":
                self.chunks.append(InlineReference(chunk))
            elif "value" in chunk and kind is None:
                self.chunks.append(TextChunk(chunk))
            else:
                it.enqueue(chunk)
                break

    def render(self):
        return (
            ''.join( 
                map(
                    lambda chunk: chunk.render(),
                    self.chunks
                ))
            ).split('\n')
        
class TextChunk:
    def __init__(self, doc):
        self.text = doc["value"]
    def render(self):
        return self.text

class CopilotFileEdit:
    def __init__(self, it):
        self.chunks = []
        for obj in it:
            self.chunks.append(obj)
            if (
                obj.get("kind", None) == "textEditGroup"
                and obj["done"] == True
            ): break

        
        editGroups = filter(
            lambda chunk: chunk.get("kind", None) == "textEditGroup",
            self.chunks
        )
        
        self.fileEdits = defaultdict(list)
        for group in editGroups:
            chunks = [
                doc["text"] 
                for chunk in group["edits"]
                    for doc in chunk
            ]
            self.fileEdits[group["uri"]["path"]].extend(chunks)

    def render(self):

        def func(tup):
            path, edits = tup
            return flatten([
                "\nEdited `" + path + '`',
                "```",
                edits,
                "```"
            ])

        return map(
            lambda path: f"\nEdited file `{path}`",
            self.fileEdits.keys()
        )

class InlineReference:
    def __init__(self, doc):
        ref = doc["inlineReference"]
        self.text = ""
        #file reference
        if "path" in ref:
            self.text = "file `" + ref["path"] + '`'
        #symbol reference
        elif "name" in ref:
            self.text = '`' + ref["name"] + '`'
        
    def render(self):
        return self.text

In [ ]:
doc = {}
with open("/home/p/Philip.Obi/chat-renderer/agent.json", "r") as f:
    import json
    doc = json.load(f)

chat = Chat(doc)

In [ ]:
with open("chat.md", "w") as f:
    f.writelines(chat.render())

In [ ]:
class Parent:
    foo = 1
    def __init__(self):
        Parent.foo = 2
        Child()


class Child:
    def __init__(self):
        print(Parent.foo)

Parent()

In [ ]:
1+1